In [130]:
import pandas as pd
import bokeh.plotting as bpl
import bokeh.models as bmd
import bokeh.layouts as bly
import bokeh.transform as btr
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
from datetime import datetime 

In [126]:
data = pd.read_csv("http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip", compression = "zip", encoding = "latin1")

In [127]:
pio.templates.default = "plotly_white"
pio.templates
print(np.__version__)


1.19.3


In [128]:
data 

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-01-02,1468a5,1,4,5,1,15,5,18,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
1,2021-01-02,043f64,2,4,9,2,9,9,10,2,...,99,1,1,2,97,3,99,MÃ©xico,97,1
2,2021-01-02,0e07d8,1,4,15,2,15,15,104,2,...,99,1,1,2,97,3,99,MÃ©xico,97,2
3,2021-01-02,13757c,1,12,15,1,15,15,106,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
4,2021-01-02,002371,1,4,3,2,3,3,8,1,...,99,1,1,2,97,3,99,MÃ©xico,97,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3662234,2021-01-02,363a18,2,12,9,2,9,9,8,1,...,1,1,3,1,2,6,99,MÃ©xico,97,97
3662235,2021-01-02,38063a,2,12,1,1,1,1,1,1,...,1,1,3,2,97,6,99,MÃ©xico,97,97
3662236,2021-01-02,2d35a2,2,12,9,2,15,9,7,1,...,2,1,3,2,97,6,99,MÃ©xico,97,97
3662237,2021-01-02,25136a,2,12,9,1,9,9,7,1,...,1,2,97,1,2,7,99,MÃ©xico,97,97


In [144]:
last_update = data["FECHA_ACTUALIZACION"].iloc[0]
date_last_update = datetime.strptime(last_update, '%Y-%m-%d')

if date_last_update.day < 10:
    day = "0" + str(date_last_update.day) 
else:
    day = str(date_last_update.day) 

if date_last_update.month < 10:
    month = "0" + str(date_last_update.month)
else:
    month = str(date_last_update.month)

str_last_update = day + "/" + month + "/" + str(date_last_update.year) 
str_last_update

'02/01/2021'

In [64]:
data[["FECHA_ACTUALIZACION", "FECHA_SINTOMAS","FECHA_INGRESO","FECHA_DEF"]] #COLUMNA QUE ME INTERESA DataFrame de una columna -----> Serie

,FECHA_ACTUALIZACION,FECHA_SINTOMAS,FECHA_INGRESO,FECHA_DEF
0,2021-01-01,2020-03-12,2020-03-18,9999-99-99
1,2021-01-01,2020-03-27,2020-04-02,9999-99-99
2,2021-01-01,2020-03-26,2020-03-26,2020-03-30
3,2021-01-01,2020-03-28,2020-03-28,2020-04-02
4,2021-01-01,2020-03-27,2020-03-31,9999-99-99
...,...,...,...,...
3653592,2021-01-01,2020-12-29,2020-12-29,9999-99-99
3653593,2021-01-01,2020-12-29,2020-12-29,9999-99-99
3653594,2021-01-01,2020-12-30,2020-12-30,9999-99-99
3653595,2021-01-01,2020-12-30,2020-12-30,9999-99-99


In [65]:
data[['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN']]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN
0,2021-01-01,071735,2
1,2021-01-01,1468a5,1
2,2021-01-01,043f64,2
3,2021-01-01,0e07d8,1
4,2021-01-01,13757c,1
...,...,...,...
3653592,2021-01-01,3a8390,2
3653593,2021-01-01,3acc33,2
3653594,2021-01-01,3f77c9,2
3653595,2021-01-01,32724b,2


In [66]:
data.loc[10] #INFORMACION DE LA FILA 10

FECHA_ACTUALIZACION      2021-01-01
ID_REGISTRO                  12d93f
ORIGEN                            1
SECTOR                            4
ENTIDAD_UM                       23
SEXO                              2
ENTIDAD_NAC                      23
ENTIDAD_RES                      23
MUNICIPIO_RES                     5
TIPO_PACIENTE                     1
FECHA_INGRESO            2020-03-24
FECHA_SINTOMAS           2020-03-20
FECHA_DEF                9999-99-99
INTUBADO                         97
NEUMONIA                          1
EDAD                             46
NACIONALIDAD                      1
EMBARAZO                         97
HABLA_LENGUA_INDIG                2
INDIGENA                          2
DIABETES                          1
EPOC                              2
ASMA                              2
INMUSUPR                          2
HIPERTENSION                      2
OTRA_COM                          2
CARDIOVASCULAR                    2
OBESIDAD                    

In [67]:
data.loc[[1,3,6]]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
1,2021-01-01,1468a5,1,4,5,1,15,5,18,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
3,2021-01-01,0e07d8,1,4,15,2,15,15,104,2,...,99,1,1,2,97,3,99,MÃ©xico,97,2
6,2021-01-01,11fb00,1,12,9,2,9,9,15,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97


In [68]:
#Casos negativos, sospechosos y activos
data["CLASIFICACION_FINAL"].value_counts()

7    1808703
3    1379537
6     343048
1      55447
5      54828
4       9833
2       2201
Name: CLASIFICACION_FINAL, dtype: int64

In [69]:
#Casos negativos ---> 7
condicion_negativos = data["CLASIFICACION_FINAL"] == 7

In [70]:
condicion_negativos.value_counts()

False    1844894
True     1808703
Name: CLASIFICACION_FINAL, dtype: int64

In [71]:
 negativos = data[condicion_negativos] #Filtrando el data FRAME para casos negativos
 print("Casos Negativos =",len(negativos))

Casos Negativos = 1808703


In [72]:
# 1,2,3 CASOS CONFIRMADOS
confirmados = data[data["CLASIFICACION_FINAL"].isin([1,2,3])]
print("Casos confirmados =",len(confirmados))

Casos confirmados = 1437185


In [73]:
confirmados["CLASIFICACION_FINAL"].value_counts()

3    1379537
1      55447
2       2201
Name: CLASIFICACION_FINAL, dtype: int64

In [74]:
#CASOS SORPECHOSOS 6 --> sospechosos sin muestra", "sospechosos con posibilidad de resultado" y "sospechosos sin posibilidad de resultado".
sospechosos = data[data["CLASIFICACION_FINAL"] == 6]
print("Sospechosos Totales = ", len(sospechosos))

Sospechosos Totales =  343048


In [75]:
sospechosos_sin_muestra=data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==97)]
#sospechosos_sin_muestra
print("Sospechosos sin muestra =",len(sospechosos_sin_muestra))

Sospechosos sin muestra = 224010


In [76]:
sospechosos_con_posibilidad =data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==3)]
#sospechosos_sin_muestra
print("Sospechosos con posibilidad =",len(sospechosos_con_posibilidad))

Sospechosos con posibilidad = 42227


In [77]:
sospechosos_sin_posibilidad=data[(data["CLASIFICACION_FINAL"].isin([6,5,4]))&(data["RESULTADO_LAB"].isin([4]))]
print("Sospechosos sin posibilidad =", len(sospechosos_sin_posibilidad))

Sospechosos sin posibilidad = 141472


In [78]:
#Buscando defunciones 
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [79]:
#data["FECHA_DEF"].value_counts()

In [80]:
defunciones = confirmados[confirmados["FECHA_DEF"] != "9999-99-99"]
print("Defunciones =", len(defunciones))


Defunciones = 126507


In [81]:
grupos = confirmados.groupby("FECHA_INGRESO")  #agrupando por fecha
confirmados_por_fecha = grupos.size().to_frame("confirmados").reset_index()   #INICIALMENTE grupos.size() es una serie con la fecha como ind
confirmados_por_fecha

,FECHA_INGRESO,confirmados
0,2020-01-13,1
1,2020-01-28,1
2,2020-01-29,1
3,2020-02-10,1
4,2020-02-20,1
...,...,...
309,2020-12-28,10332
310,2020-12-29,8691
311,2020-12-30,5927
312,2020-12-31,3340


In [82]:
confirmados_por_fecha.dtypes

FECHA_INGRESO    object
confirmados       int64
dtype: object

In [83]:
confirmados_por_fecha["FECHA_INGRESO"][:3]

0    2020-01-13
1    2020-01-28
2    2020-01-29
Name: FECHA_INGRESO, dtype: object

In [84]:
confirmados_por_fecha["fecha_ingreso"] = pd.to_datetime(confirmados_por_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")
confirmados_por_fecha

,FECHA_INGRESO,confirmados,fecha_ingreso
0,2020-01-13,1,2020-01-13
1,2020-01-28,1,2020-01-28
2,2020-01-29,1,2020-01-29
3,2020-02-10,1,2020-02-10
4,2020-02-20,1,2020-02-20
...,...,...,...
309,2020-12-28,10332,2020-12-28
310,2020-12-29,8691,2020-12-29
311,2020-12-30,5927,2020-12-30
312,2020-12-31,3340,2020-12-31


In [85]:
confirmados_por_fecha.dtypes

FECHA_INGRESO            object
confirmados               int64
fecha_ingreso    datetime64[ns]
dtype: object

In [86]:
defunciones_por_fecha = defunciones.groupby("FECHA_DEF").size().to_frame("defunciones").reset_index()
defunciones_por_fecha["fecha_defuncion"] = pd.to_datetime(defunciones_por_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_fecha


,FECHA_DEF,defunciones,fecha_defuncion
0,2020-03-03,1,2020-03-03
1,2020-03-13,1,2020-03-13
2,2020-03-17,1,2020-03-17
3,2020-03-18,2,2020-03-18
4,2020-03-20,1,2020-03-20
...,...,...,...
286,2020-12-28,530,2020-12-28
287,2020-12-29,480,2020-12-29
288,2020-12-30,315,2020-12-30
289,2020-12-31,71,2020-12-31


In [87]:
sospechosos_por_fecha = sospechosos.groupby("FECHA_SINTOMAS").size().to_frame("sospechosos").reset_index()
sospechosos_por_fecha["fecha_sintomas"] = pd.to_datetime(sospechosos_por_fecha["FECHA_SINTOMAS"], format = "%Y-%m-%d")


In [88]:
bpl.output_notebook()

Loading BokehJS ...

In [119]:
# Ploting each line in different figures ...
def plotCases(abscisa, ordenada, x_title, y_title, g_title, paint ):
    fig_1 = px.line( x = abscisa, y = ordenada, title = g_title, width = 1500, height = 600) # labels parameter can be added with a dict

    fig_1.update_traces(marker = dict(color = paint), line = dict(color = paint, width = 2))
    fig_1.update_xaxes(showgrid = True, title = x_title)
    fig_1.update_yaxes(showgrid = True, title = y_title)
    fig_1.show()
    
plotCases(confirmados_por_fecha["fecha_ingreso"], confirmados_por_fecha["confirmados"], 
        "Fecha de registro", "Casos confirmados", "Casos de COVID-19 en México","#b4004e")
plotCases(defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones"], 
        "Fecha de defunción", "Defunciones", "Defunciones por COVID-19 en México" , "black")
# plotCases(sospechosos_por_fecha["fecha_sintomas"],sospechosos_por_fecha["sospechosos"], 
#         "Fecha del inicio de sintomas", "Casos sospechosos", "Casos sospechosos de COVID-19 en México", "#c67c00")


In [120]:
# Computing the comulative cases ...
def commulative_cases(data):
    data_vec = np.array(data)
    data_acumulados = np.cumsum(data_vec)    
    return pd.Series(data_acumulados)                           # Numpy array to pandas Serie

# Commulative confirmed cases ....
confirmados_por_fecha["casos_acumulados"] = commulative_cases(confirmados_por_fecha["confirmados"])
# Commulative Deaths  ....
defunciones_por_fecha["defunciones_acumuladas"] = commulative_cases(defunciones_por_fecha["defunciones"])

In [91]:
#Computing the average with the actual day plus the 6 previous days
def average_of_seven_days(serie):
    data_vect = np.array(serie)
    average = np.zeros(data_vect.size)
    counter = 0
    init = 0

    for i in range(data_vect.size):
        if i >= 6:
            average[counter] = np.mean(data_vect[init:i+1])
            init += 1
        else:
            average[counter] = 1/7 * np.sum(data_vect[0:i+1])

        counter += 1

    return  pd.Series(np.round(average))   #Return a pandas Serie with rounded values

In [92]:
# confirmados_por_fecha["casos_promedio"] =  average_of_seven_days(confirmados_por_fecha["confirmados"])
# defunciones_por_fecha["defunciones_promedio"] = average_of_seven_days(defunciones_por_fecha["defunciones"])

In [93]:

# fig_bar1_conf = px.bar(x =confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
    
#                     labels= dict(x = "Fecha de registro", y = "Nuevos casos confirmados"), title="Nuevos casos de COVID-19 reportados diariamente", width = 900)
#fig_bar1_conf.update_traces( marker = dict(color = "#ff1744", opacity = 0.6))
# fig_bar2_def = px.bar(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"],
#                         labels = dict(x = "Fecha de defunción", y = "Defunciones"), title = "Nuevas defunciones por COVID-19 reportadas diariamente", width = 900)

# fig_bar2_def.update_traces(marker = dict(color = "black"))

# Bar plots for confirmed and death cases ...
def show_confirmed_cases(cases, region):
    fig_bar1 = go.Figure()
    fig_bar1.add_trace(go.Bar(x =cases["fecha_ingreso"], y = cases["confirmados"], 
                                marker = dict(color = "#ff1744", opacity = 0.6 ), name = "Nuevos casos"))
    #Average of seven days curve ...                             
    fig_bar1.add_trace(go.Scatter(x = cases["fecha_ingreso"], y =  average_of_seven_days(cases["confirmados"]),
                                        mode = "lines", line = dict(color = "#b71c1c", width = 2.5), name = "Promedio de 7 días"))
    #custom layout
    fig_bar1.update_xaxes(title = "Fecha de registro",spikethickness= .4)
    fig_bar1.update_yaxes(title = "Nuevos casos confirmados")
    fig_bar1.update_layout(width = 900, hovermode = "x unified", title = "Nuevos casos de COVID-19 reportados diariamente en " + region)
    #Showing figure ... 
    fig_bar1.show()

def show_deaths(deaths,region):
    # Death cases ...
    fig_bar2 = go.Figure()
    fig_bar2.add_trace(go.Bar(x = deaths["fecha_defuncion"], y = deaths["defunciones"],
                                marker = dict(color = "#546e7a", opacity = 0.75), name = "Nuevas defunciones"))
    #Average of seven days curve ...                         
    fig_bar2.add_trace(go.Scatter(x = deaths["fecha_defuncion"], y =average_of_seven_days(deaths["defunciones"]),
                                        mode = "lines", line = dict(color = "black", width = 2.5), name = "Promedio de 7 días"))
    #custom layout
    fig_bar2.update_xaxes(title = "Fecha de registro",spikethickness= .4)
    fig_bar2.update_yaxes(title = "Nuevas defunciones confirmadas")
    fig_bar2.update_layout(width = 900, hovermode = "x unified", title = "Nuevas defunciones de COVID-19 reportadas diariamente en " + region)
    #Showing figure ...
    fig_bar2.show()

show_confirmed_cases(confirmados_por_fecha, "México")
show_deaths(defunciones_por_fecha,"México")

In [94]:
# Ploting the commulative confirmed and deaths cases ...

def acum_plot(x_conf_data, y_conf_data, x_def_data, y_def_data, region ):
    fig_acum = go.Figure()

    fig_acum.add_trace(go.Scatter(x = x_conf_data, y = y_conf_data,
                    mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

    fig_acum.add_trace(go.Scatter(x = x_def_data, y = y_def_data,
                    mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))

    fig_acum.update_xaxes(title = "Fecha de registro")
    fig_acum.update_yaxes(title = "Número de casos acumulados")
    fig_acum.update_layout(title = "Casos acumulados en " + region )
    fig_acum.show()


# fig_acum = go.Figure()

# fig_acum.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["casos_acumulados"],
#                     mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

# fig_acum.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones_acumuladas"],
#                     mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))



# fig_acum.update_xaxes(title = "Fecha de registro")
# fig_acum.update_yaxes(title = "Número de casos acumulados")
# fig_acum.update_layout(title = "Casos acumulados en México")
# fig_acum.show()

acum_plot(confirmados_por_fecha["fecha_ingreso"],confirmados_por_fecha["casos_acumulados"],
            defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones_acumuladas"], region = "México" )


In [95]:

# Ploting each line in the same figure ...

# fig_2 = px.line(confirmados_por_fecha, x = "fecha_sintomas", y = "confirmados", width = 1000, height= 300,
#                  labels= dict(fecha_sintomas = "Fecha", confirmados = "Número de personas", name = "Casos confirmados"))

fig_2 = go.Figure()

fig_2.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
                mode = "lines",name = "Casos confirmados", line = dict(color = "#b4004e")) )
fig_2.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"], 
                mode = "lines",name = "Defunciones", line = dict(color = "black") ) )
fig_2.add_trace(go.Scatter(x = sospechosos_por_fecha["fecha_sintomas"], y = sospechosos_por_fecha["sospechosos"],
                mode = "lines", name = "Casos sospechosos", line = dict(color = "#c67c00")))

fig_2.update_xaxes(showgrid = True, title = "Fecha")
fig_2.update_yaxes(showgrid = True , title = "Número de personas")
fig_2.update_layout( width = 1200, height= 400 , title = "COVID-19 México")
fig_2.show()

In [96]:
# Distribution of cases per state ...................................
confirmados

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-01-01,071735,2,9,21,2,21,21,114,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
1,2021-01-01,1468a5,1,4,5,1,15,5,18,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
2,2021-01-01,043f64,2,4,9,2,9,9,10,2,...,99,1,1,2,97,3,99,MÃ©xico,97,1
3,2021-01-01,0e07d8,1,4,15,2,15,15,104,2,...,99,1,1,2,97,3,99,MÃ©xico,97,2
4,2021-01-01,13757c,1,12,15,1,15,15,106,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3653577,2021-01-01,3da0ef,2,12,9,1,9,9,13,1,...,2,2,97,1,1,3,99,MÃ©xico,97,97
3653578,2021-01-01,2887cf,2,12,9,1,9,9,5,1,...,2,2,97,1,1,3,99,MÃ©xico,97,97
3653582,2021-01-01,221490,1,12,9,1,9,9,7,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97
3653585,2021-01-01,374eb2,2,12,9,1,15,9,10,1,...,2,2,97,1,2,1,99,MÃ©xico,97,97


In [97]:
#Grouping by state ENTIDAD_RES
confirmados_por_estado = confirmados.groupby("ENTIDAD_RES").size().to_frame("Casos confirmados").reset_index()
confirmados_por_estado

,ENTIDAD_RES,Casos confirmados
0,1,17021
1,2,34889
2,3,17820
3,4,7273
4,5,49685
5,6,7949
6,7,8293
7,8,36517
8,9,334137
9,10,25248


In [98]:
# Cargando el diccionario de estados
dics = pd.read_excel("../diccionarios/201128 Catalogos_.xlsx", sheet_name = None)
dics.keys()

dict_keys(['Catálogo ORIGEN', 'Catálogo SECTOR', 'Catálogo SEXO', 'Catálogo TIPO_PACIENTE', 'Catálogo SI_NO', 'Catálogo NACIONALIDAD', 'Catálogo RESULTADO_LAB', 'Catálogo RESULTADO_ANTIGENO', 'Catálogo CLASIFICACION_FINAL', 'Catálogo de ENTIDADES', 'Catálogo MUNICIPIOS'])

In [99]:
dic_estados = dics["Catálogo de ENTIDADES"]
dic_estados

,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,AGUASCALIENTES,AS
1,2,BAJA CALIFORNIA,BC
2,3,BAJA CALIFORNIA SUR,BS
3,4,CAMPECHE,CC
4,5,COAHUILA DE ZARAGOZA,CL
5,6,COLIMA,CM
6,7,CHIAPAS,CS
7,8,CHIHUAHUA,CH
8,9,CIUDAD DE MÉXICO,DF
9,10,DURANGO,DG


In [100]:
confirmados_por_entidad = pd.merge(confirmados_por_estado, dic_estados, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD" ) # left outer join
confirmados_por_entidad

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,17021,1,AGUASCALIENTES,AS
1,2,34889,2,BAJA CALIFORNIA,BC
2,3,17820,3,BAJA CALIFORNIA SUR,BS
3,4,7273,4,CAMPECHE,CC
4,5,49685,5,COAHUILA DE ZARAGOZA,CL
5,6,7949,6,COLIMA,CM
6,7,8293,7,CHIAPAS,CS
7,8,36517,8,CHIHUAHUA,CH
8,9,334137,9,CIUDAD DE MÉXICO,DF
9,10,25248,10,DURANGO,DG


In [101]:
confirmados_por_entidad.dtypes

ENTIDAD_RES            int64
Casos confirmados      int64
CLAVE_ENTIDAD          int64
ENTIDAD_FEDERATIVA    object
ABREVIATURA           object
dtype: object

In [102]:
# Bar plot of the confirmed cases by stte
confirmados_por_entidad.sort_values("Casos confirmados", ascending = False, inplace = True ) # Ordenando por número de casos de forma descendente 
fig_bar_edo = px.bar( x = confirmados_por_entidad["ENTIDAD_FEDERATIVA"], y = confirmados_por_entidad["Casos confirmados"],
                        labels = dict(x = "Entidad Federativa", y = "Número de casos confirmados"), title = "Número de casos confirmados por estado")


fig_bar_edo.update_xaxes(type='category', tickfont=dict(size = 10))
fig_bar_edo.update_traces(marker = dict(color = "#b71c1c"), opacity = 0.8)
fig_bar_edo.show()

In [103]:
poblaciones = pd.read_csv("http://www.conapo.gob.mx/work/models/CONAPO/Datos_Abiertos/Proyecciones2018/pob_mit_proyecciones.csv", encoding="latin1")
poblaciones

,RENGLON,AÑO,ENTIDAD,CVE_GEO,EDAD,SEXO,POBLACION
0,1,1950,República Mexicana,0,0,Hombres,572103
1,2,1950,República Mexicana,0,0,Mujeres,559162
2,3,1950,República Mexicana,0,1,Hombres,514540
3,4,1950,República Mexicana,0,1,Mujeres,505269
4,5,1950,República Mexicana,0,2,Hombres,478546
...,...,...,...,...,...,...,...
592455,592456,2050,Zacatecas,32,107,Mujeres,4
592456,592457,2050,Zacatecas,32,108,Hombres,1
592457,592458,2050,Zacatecas,32,108,Mujeres,2
592458,592459,2050,Zacatecas,32,109,Hombres,0


In [104]:
poblacion_2020 = poblaciones[poblaciones["AÑO"] == 2020]

In [105]:
grupos_pob = poblacion_2020.groupby(["CVE_GEO","ENTIDAD"])
poblacion_tot_estado = grupos_pob["POBLACION"].sum().to_frame("POBLACION").reset_index()   # Sum operation in the POBLACION column of each group

In [106]:
datos_por_estado = pd.merge(confirmados_por_entidad, poblacion_tot_estado, how = "left", left_on = "CLAVE_ENTIDAD", right_on = "CVE_GEO" )
datos_por_estado.columns

Index(['ENTIDAD_RES', 'Casos confirmados', 'CLAVE_ENTIDAD',
       'ENTIDAD_FEDERATIVA', 'ABREVIATURA', 'CVE_GEO', 'ENTIDAD', 'POBLACION'],
      dtype='object')

In [107]:
# Rate of confirmed case by 100,000 people 
datos_por_estado["TASA DE CASOS POR 100K"] = 100000 * datos_por_estado["Casos confirmados"]/datos_por_estado["POBLACION"]
datos_por_estado.sort_values("TASA DE CASOS POR 100K", ascending = False, inplace = True)
datos_por_estado.reset_index(drop = True, inplace = True)
datos_por_estado

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA,CVE_GEO,ENTIDAD,POBLACION,TASA DE CASOS POR 100K
0,9,334137,9,CIUDAD DE MÉXICO,DF,9,Ciudad de México,9018645,3704.957896
1,3,17820,3,BAJA CALIFORNIA SUR,BS,3,Baja California Sur,804708,2214.467857
2,27,44592,27,TABASCO,TC,27,Tabasco,2572287,1733.554615
3,26,52146,26,SONORA,SR,26,Sonora,3074745,1695.945517
4,5,49685,5,COAHUILA DE ZARAGOZA,CL,5,Coahuila,3218720,1543.626038
5,19,83209,19,NUEVO LEÓN,NL,19,Nuevo León,5610153,1483.185931
6,22,32582,22,QUERÉTARO,QT,22,Querétaro,2279637,1429.262641
7,24,39968,24,SAN LUIS POTOSÍ,SP,24,San Luis Potosí,2866142,1394.487782
8,10,25248,10,DURANGO,DG,10,Durango,1868996,1350.885716
9,11,83520,11,GUANAJUATO,GT,11,Guanajuato,6228175,1341.002782


In [108]:
# Bar plot of the rate of confirmed cases per 100k persons 
fig_bar_cases_rate = px.bar(x = datos_por_estado["ENTIDAD"], y = datos_por_estado["TASA DE CASOS POR 100K"],
                            labels = dict(y = "Casos confirmados / 100k personas" , x = "") ,title = "Tasa de casos confirmados por cada 100,000 personas")


fig_bar_cases_rate.update_traces(marker = dict(color = "#ff616f"))
#fig_bar_cases_rate.update_xaxes(tickfont = dict(size = 11))
fig_bar_cases_rate.show()

In [109]:
# Groping by state ENTIDAD_RES and FECHA_INGRESO
confirmados_por_estado_fecha = confirmados.groupby(["ENTIDAD_RES","FECHA_INGRESO"]).size().to_frame("confirmados").reset_index()
confirmados_por_estado_fecha["fecha_ingreso"] = pd.to_datetime(confirmados_por_estado_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")


confirmados_por_estado_fecha

,ENTIDAD_RES,FECHA_INGRESO,confirmados,fecha_ingreso
0,1,2020-02-20,1,2020-02-20
1,1,2020-03-14,1,2020-03-14
2,1,2020-03-17,1,2020-03-17
3,1,2020-03-18,3,2020-03-18
4,1,2020-03-19,1,2020-03-19
...,...,...,...,...
9202,32,2020-12-27,37,2020-12-27
9203,32,2020-12-28,146,2020-12-28
9204,32,2020-12-29,144,2020-12-29
9205,32,2020-12-30,107,2020-12-30


In [110]:
defunciones_por_estado_fecha = defunciones.groupby(["ENTIDAD_RES", "FECHA_DEF"]).size().to_frame("defunciones").reset_index()
defunciones_por_estado_fecha["fecha_defuncion"] = pd.to_datetime(defunciones_por_estado_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_estado_fecha

,ENTIDAD_RES,FECHA_DEF,defunciones,fecha_defuncion
0,1,2020-04-10,1,2020-04-10
1,1,2020-04-13,1,2020-04-13
2,1,2020-04-17,1,2020-04-17
3,1,2020-04-19,1,2020-04-19
4,1,2020-04-22,1,2020-04-22
...,...,...,...,...
8146,32,2020-12-26,4,2020-12-26
8147,32,2020-12-27,7,2020-12-27
8148,32,2020-12-28,9,2020-12-28
8149,32,2020-12-29,8,2020-12-29


In [111]:
entidad_clave = datos_por_estado["ENTIDAD"].to_frame()
entidad_clave["CLAVE_ENTIDAD"] = datos_por_estado["CLAVE_ENTIDAD"]
# Merge between entidad_cave and confirmados_por_estado_fecha
confirmados_por_estado_fecha_clave = pd.merge(confirmados_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")
confirmados_por_estado_fecha_clave


,ENTIDAD_RES,FECHA_INGRESO,confirmados,fecha_ingreso,ENTIDAD,CLAVE_ENTIDAD
0,1,2020-02-20,1,2020-02-20,Aguascalientes,1
1,1,2020-03-14,1,2020-03-14,Aguascalientes,1
2,1,2020-03-17,1,2020-03-17,Aguascalientes,1
3,1,2020-03-18,3,2020-03-18,Aguascalientes,1
4,1,2020-03-19,1,2020-03-19,Aguascalientes,1
...,...,...,...,...,...,...
9202,32,2020-12-27,37,2020-12-27,Zacatecas,32
9203,32,2020-12-28,146,2020-12-28,Zacatecas,32
9204,32,2020-12-29,144,2020-12-29,Zacatecas,32
9205,32,2020-12-30,107,2020-12-30,Zacatecas,32


In [112]:
# Merge betweem entidad_clave and defunciones_por_estado_fecha
defunciones_por_estado_fecha_clave = pd.merge(defunciones_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")
defunciones_por_estado_fecha_clave

,ENTIDAD_RES,FECHA_DEF,defunciones,fecha_defuncion,ENTIDAD,CLAVE_ENTIDAD
0,1,2020-04-10,1,2020-04-10,Aguascalientes,1
1,1,2020-04-13,1,2020-04-13,Aguascalientes,1
2,1,2020-04-17,1,2020-04-17,Aguascalientes,1
3,1,2020-04-19,1,2020-04-19,Aguascalientes,1
4,1,2020-04-22,1,2020-04-22,Aguascalientes,1
...,...,...,...,...,...,...
8146,32,2020-12-26,4,2020-12-26,Zacatecas,32
8147,32,2020-12-27,7,2020-12-27,Zacatecas,32
8148,32,2020-12-28,9,2020-12-28,Zacatecas,32
8149,32,2020-12-29,8,2020-12-29,Zacatecas,32


In [113]:
clave_estado = 11
confirmados_mich = confirmados_por_estado_fecha_clave[confirmados_por_estado_fecha_clave["ENTIDAD_RES"] == clave_estado]
print("Total de casos confirmados = ",confirmados_mich["confirmados"].sum())
confirmados_mich.reset_index(drop = True, inplace = True)
confirmados_mich["confirmados_acumulados"] = commulative_cases(confirmados_mich["confirmados"]) # Accumulated confirmed cases in Mich.
confirmados_mich

Total de casos confirmados =  83520


,ENTIDAD_RES,FECHA_INGRESO,confirmados,fecha_ingreso,ENTIDAD,CLAVE_ENTIDAD,confirmados_acumulados
0,11,2020-03-13,1,2020-03-13,Guanajuato,11,1
1,11,2020-03-15,2,2020-03-15,Guanajuato,11,3
2,11,2020-03-16,2,2020-03-16,Guanajuato,11,5
3,11,2020-03-18,1,2020-03-18,Guanajuato,11,6
4,11,2020-03-19,1,2020-03-19,Guanajuato,11,7
...,...,...,...,...,...,...,...
287,11,2020-12-27,271,2020-12-27,Guanajuato,11,81927
288,11,2020-12-28,749,2020-12-28,Guanajuato,11,82676
289,11,2020-12-29,630,2020-12-29,Guanajuato,11,83306
290,11,2020-12-30,165,2020-12-30,Guanajuato,11,83471


In [114]:
defunciones_mich = defunciones_por_estado_fecha_clave[defunciones_por_estado_fecha_clave["ENTIDAD_RES"] == clave_estado ]
print("Total de defunciones acumuladas = ", defunciones_mich["defunciones"].sum())
defunciones_mich.reset_index(drop = True, inplace = True)     
defunciones_mich["defunciones_acumuladas"] = commulative_cases(defunciones_mich["defunciones"])
defunciones_mich

Total de defunciones acumuladas =  5365


,ENTIDAD_RES,FECHA_DEF,defunciones,fecha_defuncion,ENTIDAD,CLAVE_ENTIDAD,defunciones_acumuladas
0,11,2020-04-05,2,2020-04-05,Guanajuato,11,2
1,11,2020-04-07,1,2020-04-07,Guanajuato,11,3
2,11,2020-04-09,1,2020-04-09,Guanajuato,11,4
3,11,2020-04-10,1,2020-04-10,Guanajuato,11,5
4,11,2020-04-14,2,2020-04-14,Guanajuato,11,7
...,...,...,...,...,...,...,...
255,11,2020-12-27,28,2020-12-27,Guanajuato,11,5251
256,11,2020-12-28,35,2020-12-28,Guanajuato,11,5286
257,11,2020-12-29,48,2020-12-29,Guanajuato,11,5334
258,11,2020-12-30,23,2020-12-30,Guanajuato,11,5357


In [115]:
#
acum_plot(confirmados_mich["fecha_ingreso"],confirmados_mich["confirmados_acumulados"],
            defunciones_mich["fecha_defuncion"], defunciones_mich["defunciones_acumuladas"], region = confirmados_mich["ENTIDAD"].iloc[0] )
print("Total de defunciones acumuladas = ", defunciones_mich["defunciones"].sum())
print("Total de casos confirmados = ",confirmados_mich["confirmados"].sum())

Total de defunciones acumuladas =  5365
Total de casos confirmados =  83520


In [116]:
show_confirmed_cases(confirmados_mich, region = confirmados_mich["ENTIDAD"].iloc[0])
show_deaths(defunciones_mich, region = confirmados_mich["ENTIDAD"].iloc[0])

In [117]:
jhu_data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", encoding="utf-8")
mexico_jhu_data = jhu_data[jhu_data["Country/Region"] == "Mexico"]
mexico_jhu_data

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21
180,NaN,Mexico,23.6345,-102.5528,0,0,0,0,0,0,...,1350079,1362564,1372243,1377217,1383434,1389430,1401529,1413935,1426094,1437185


In [118]:
jhu_daily_reports = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/12-29-2020.csv", encoding="utf-8")
mexico_daily_report = jhu_daily_reports[jhu_daily_reports["Country_Region"] == "Mexico"]
mexico_daily_report

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
386,NaN,NaN,Aguascalientes,Mexico,2020-12-30 05:22:34,21.8853,-102.2916,16805,1349,0,15456.0,"Aguascalientes, Mexico",1171.378086,8.027373
387,NaN,NaN,Baja California,Mexico,2020-12-30 05:22:34,30.8406,-115.2838,33776,5388,0,28388.0,"Baja California, Mexico",929.222189,15.952155
388,NaN,NaN,Baja California Sur,Mexico,2020-12-30 05:22:34,26.0444,-111.6661,17280,763,0,16517.0,"Baja California Sur, Mexico",2147.362770,4.415509
389,NaN,NaN,Campeche,Mexico,2020-12-30 05:22:34,19.8301,-90.5349,7233,964,0,6269.0,"Campeche, Mexico",722.853999,13.327803
390,NaN,NaN,Chiapas,Mexico,2020-12-30 05:22:34,16.7569,-93.1292,8228,1198,0,7030.0,"Chiapas, Mexico",143.585917,14.560039
391,NaN,NaN,Chihuahua,Mexico,2020-12-30 05:22:34,28.6330,-106.0691,36032,4387,0,31645.0,"Chihuahua, Mexico",947.839622,12.175289
392,NaN,NaN,Ciudad de Mexico,Mexico,2020-12-30 05:22:34,19.4326,-99.1332,320251,16385,0,303866.0,"Ciudad de Mexico, Mexico",3550.987981,5.116299
393,NaN,NaN,Coahuila,Mexico,2020-12-30 05:22:34,27.0587,-101.7068,48772,4119,0,44653.0,"Coahuila, Mexico",1515.260725,8.445420
394,NaN,NaN,Colima,Mexico,2020-12-30 05:22:34,19.1223,-104.0072,7832,795,0,7037.0,"Colima, Mexico",997.512587,10.150664
395,NaN,NaN,Durango,Mexico,2020-12-30 05:22:34,24.5593,-104.6588,24893,1521,0,23372.0,"Durango, Mexico",1331.891561,6.110151
